In [15]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)

In [17]:
N_EPOCHS = 50
N_TRIALS = 50
BATCH_SIZE = 512

CONFIG = config_tests["assessment_proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [18]:
x, y = utils.load_cifar_data("../data/cifar-10-batches-py/")
x /= 255

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.6, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [19]:
train_size, len(x_val), len(x_test)

(20000, 15000, 15000)

In [20]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(loss=loss, metrics=["accuracy"])
assessment_model(x_train[:1])
assessment_model.load_weights("../models/assessment_model.weights.h5")

/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 38 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [21]:
assessment_model.evaluate(x_test, y_test)

469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5517 - loss: 1.2630


[1.2574479579925537, 0.5510666370391846]

In [ ]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

samples_proba = utils.calculate_proba2(
    assessment_model, x_train_sorted, y_train_sorted, counts
)

In [ ]:
model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["test_model"])
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
### model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [ ]:
model_scores = []

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)
        
    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
        
        samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            # validation_data=(x_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=0,
        )
        
        if CONFIG["progressive"]:
            samples_proba = utils.calculate_proba2(
                model, x_train_sorted, y_train_sorted, counts # , CONFIG["negative_loss"]
            )
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
    model_scores.append(accuracy)
    print("Mean:", np.mean(model_scores))

In [ ]:
np.median(model_scores)

In [ ]:
CONFIG

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.subplot(131)
# plt.boxplot(results["test_model_1"])
# plt.ylim([0.63, 0.65])
# plt.subplot(132)
# plt.boxplot(ass_m["test_model_1"])
# plt.ylim([0.63, 0.65])
# plt.subplot(133)
# plt.boxplot(random_m["test_model_1"])
# plt.ylim([0.63, 0.65])
# plt.show()

In [ ]:
abc = [0.6460999846458435,
 0.705299973487854,
 0.6814000010490417,
 0.6901000142097473,
 0.6704000234603882,
 0.679099977016449,
 0.6834999918937683,
 0.6952000260353088,
 0.6812999844551086,
 0.6843000054359436,
 0.7019000053405762,
 0.6528000235557556,
 0.6559000015258789,
 0.6805999875068665,
 0.5934000015258789,
 0.6704999804496765,
 0.6823999881744385,
 0.6553999781608582,
 0.6852999925613403,
 0.6736999750137329,
 0.6478000283241272,
 0.6850000023841858,
 0.5813999772071838,
 0.6521000266075134,
 0.679099977016449,
 0.6668999791145325,
 0.6913999915122986,
 0.6065000295639038,
 0.6934999823570251,
 0.6527000069618225,
 0.6782000064849854,
 0.6826000213623047,
 0.6948999762535095,
 0.6331999897956848,
 0.6876999735832214,
 0.6541000008583069,
 0.7009999752044678,
 0.6833000183105469,
 0.6970999836921692,
 0.673799991607666,
 0.6690000295639038,
 0.6815000176429749,
 0.6561999917030334,
 0.6574000120162964,
 0.6601999998092651,
 0.6599000096321106,
 0.684499979019165,
 0.6427000164985657,
 0.6690000295639038,
 0.6814000010490417]

In [ ]:
import scipy.stats
scipy.stats.ttest_ind(abc, model_scores)

In [ ]:
abc = [0.6462666392326355,
 0.6143333315849304,
 0.6458666920661926,
 0.6164666414260864,
 0.5498666763305664,
 0.659333348274231,
 0.5580000281333923,
 0.6102666854858398,
 0.6272666454315186,
 0.5816666483879089,
 0.5935333371162415,
 0.6465333104133606,
 0.6446666717529297,
 0.6413333415985107,
 0.6402666568756104,
 0.6539333462715149,
 0.627133309841156,
 0.5664666891098022,
 0.6363333463668823,
 0.5622000098228455,
 0.5845999717712402,
 0.6078000068664551,
 0.6403999924659729,
 0.5857333540916443,
 0.60753333568573,
 0.6057999730110168,
 0.5952666401863098,
 0.6287333369255066,
 0.6296666860580444,
 0.598800003528595,
 0.6316666603088379,
 0.6499999761581421,
 0.6220666766166687,
 0.5823333263397217,
 0.6305333375930786,
 0.5469333529472351,
 0.6465333104133606,
 0.623199999332428,
 0.6430000066757202,
 0.6377333402633667,
 0.5971999764442444,
 0.6179333329200745,
 0.6143333315849304,
 0.584933340549469,
 0.4405333399772644,
 0.6126000285148621,
 0.5825999975204468,
 0.6133333444595337,
 0.6164666414260864,
 0.5774666666984558]
np.median(abc)